## Final Project Submission

Please fill out:
* Student name: Claire Sarraille
* Student pace: self paced
* Scheduled project review date/time:  Monday June 14th 1pm PST
* Instructor name: Jeff Herman
* Project Repo URL: https://github.com/clairesarraille/mod1finproj/tree/main/dsc-mod-1-project-v2-1-online-ds-sp-000-master
* Blog post URL:


Set environment before running this script with: conda activate learn-env
Import datascience libraries:

In [265]:
# Import datascience libraries:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

# Import libraries for fetching and formatting data:
from tmdbv3api import TMDb, Discover, Movie
import os

# Import libraries for performing operations:
from math import ceil

# Load classes from tmdbv3api - a python library for The Movie Database (TMDb) API
tmdb = TMDb()
discover = Discover()
movie = Movie()

# Store my API key in my previously set environment variable as a tmdb.api_key string
priv_api_key = os.environ.get('TMDB_PRIVATE_API_KEY')
tmdb.api_key = priv_api_key


Preview of discover.discover_movies data:

I used this function to quickly scan a sub-set of titles & tweak variables such as:
- 'with_original_language': 'en'

I used a slightly different function to pretty-print samples of all info for a sub-set of movies. See appendix.

In [266]:
def print_sample_tmdb(m, m_r, p, y):
    print()
    print(f"Year is: {y}")
    print(f"Page is: {p +1}")
    print(f"There are {len(m[y])} movies in movies dict for {y}")
    for d in m_r[0:1]:
        print("sample original title: ", (dict(d))['original_title'])
    print()
    

Function to extract 500 movies each for years 2016-2021 inclusive, sorted by revenue descending
This line is commented out in the code below, to preserve space in the notebook:
- print_sample_tmdb(movies, movies_running, page, year)

In [267]:
# This function uses the discover.discover_movies() method from the tmdbv3api package to intake initial movie data:

# Define Constants
# Constants are usually defined on a module level and written in all capital letters with underscores separating words
N_YEAR_RANGE = range(2016, 2022)  # grab 2016-2021 data, inclusive
N_RESULT = 500  # Enter number of results desired for each year
N_RESULT_PAGE = 20  # Discover() class returns 20 results per page
# Num pages = Num results / num results per page, rounded UP using math.ceil
N_PAGES = ceil(N_RESULT / N_RESULT_PAGE)

# Instantiate empty dictionary movies to store outside of the following loops,
# because you want to use movies after loops are finished executing
movies = {}

# Function to grab data from discover.discover_movies:


def return_discover_movies():
    for year in N_YEAR_RANGE:  # iterate through years 2000-2019 inclusive
        # Keys are integers for each year
        # Values will be lists of tmdbv3api objects, converted to dictionaries
        movies[year] = []
        for page in range(0, N_PAGES):
            # For N_PAGES, create list movies_running, querying results by year, page, & sort revenue desc
            # The objects within movies_running 'tmdbv3api.as_obj.AsObj' objects, and will need to be converted to dictionaries later
            movies_running = discover.discover_movies({
                # We want to sort by revenue so that we make sure we're getting the highest grossing movies in each batch of 500
                'sort_by': 'revenue.desc',
                'primary_release_year': str(year),
                # API is 1 indexed - so we grab pages 1 to N_PAGES inclusive
                'page': str(page + 1),
                # Movies with 0 votes are likely among the most unpopular. OK to filter by at least 1 vote.
                'vote_count.gte': 1,
                # Since our studio is still in R&D mode, we'll focus on movies originally shot in English
                'with_original_language': 'en'
            })
            # For each year in our movies dictionary, store the year as a key (datatype list) in dictionary "movies"
            # Then, iterate through 'tmdbv3api.as_obj.AsObj' objects in movies_running for N_PAGES, converting each object as a dictionary
            # For each year we will have a list of dictionaries, and each dictionary is a set of information for a single movie
            # .extend used rather than .append in order to extend our list by >1 items at once
            movies[year].extend([dict(tmdb_obj)
                                for tmdb_obj in movies_running])

            # print_sample_tmdb(movies, movies_running, page, year)  # Preview original movie titles:


# Now we have our dictionary movies, with years 2000-2019 inclusive, each with 100 movies in each list from that year
# We didn't need to return movies in the definition of this function, because movies was declared before the function
return_discover_movies()


# Now that we've collected our 3,000 titles, sorted by descending revenue for each year, let's do some initial data exploration and clenaing:

In [268]:
# Sanity check there are 3,000 titles in movies dictionary
# Print ALL movie titles and ids from movies dictionary to qa_titles.txt for data-cleaning prep: manual review
movie_count = 0
for year in movies:
    for moo_vie in range(0, len(movies[year])):
        movie_count += 1
        my_string = f"{movies[year][moo_vie]['title']}       {movies[year][moo_vie]['id']}"
        with open("/Users/clairesarraille/git-repos/mod1finproj/dsc-mod-1-project-v2-1-online-ds-sp-000-master/qa_files/qa_titles.txt", "a+", encoding='utf-8') as file_object:
            # Move read cursor to the start of file.
            file_object.seek(0)
            data = file_object.read(100)
            if len(data) > 0:
                file_object.write("\n")
            file_object.write(my_string)

# Print movie counts in movies dict:
print("The total movie count returned from our return_discover_movies() function above is: ")
print(movie_count)

# Print keys in movies dict:
print()
print("The keys in our movies dictionary are:")
print(movies.keys())


The total movie count returned from our return_discover_movies() function above is: 
3000

The keys in our movies dictionary are:
dict_keys([2016, 2017, 2018, 2019, 2020, 2021])


While scanning the above list, I found the following that I'll use to develop some data clean-up methods:

- These don't look legitimate:
    - movie_02    813075
    - movie_01    813064
    - welcome_home.exe    82418
    - jji    763071

- There's a double entry for this movie:
    - Split    358364
    - Split    381288

- Not in English, this one I later caught when running the analysis to find non-ascii characters:
    - عتيج    676624


In [325]:
# Upon internet research and cross-referencing I found that they're all legitimate movies. Nice!

likely_suspects = [813075,813064,824186,763071,358364,381288]

for suspect in likely_suspects:
    m = movie.details(suspect)   
    print(m.title)
    print(m.id)
    print(m.release_date)
    print(m.overview)
    print(m.popularity)
    print()


movie_02
813075
2021-04-02
An experimental short shot in March of 2021.
0.6

movie_01
813064
2021-03-24
An experimental short shot over summer 2020.
0.6

welcome_home.exe
824186
2021-04-29
As technology accelerates, our species' collective imagination of the future grows ever more kaleidoscopic. We are all haunted by temporal distortion, perhaps no more than when we attempt to remember what the future looked like to our younger selves. As the mist of time devours our memories, the future recedes; each of us burdened by the gaping mouth of entropy. Yet, emerging technology provides a glimmer of hope; transhumanism promises a future free from mortality, disease and pain. Does our salvation lie in digital simulacra? We're here to sell you the answer to that question, for the low, low price of four hundred and seventy seconds.
0.6

jji
763071
2019-04-19
the summer breathes.
0.6

Split
358364
2016-04-07
The story of a young woman who takes an epic journey to claim her own darkness and sexua

In [269]:
# Check that API is working properly by spot-checking movie titles/ids between methods discover.discover_movies and movie.details()

# Expected data (from discover.movies)
# Bigfoot vs the Illuminati
# 784281
my_result = movie.details(784281)
print(my_result.title)
print(my_result.id)
print()

# Expected data (from discover.movies)
# Cheez-It's & Chili
# 802103
my_result = movie.details(802103)
print(my_result.title)
print(my_result.id)
print()

# Expected data (from discover.movies)
# A Tiny House Christmas
# 796438
my_result = movie.details(796438)
print(my_result.title)
print(my_result.id)
print()


Bigfoot vs the Illuminati
784281

Cheez-It's & Chili
802103

A Tiny House Christmas
796438



Our movies dictionary has keys for years 2016-2021
The value for each year is a list of dictionaries. Each dictionary is a movie.
To turn this data into a Pandas datastructure, we iterate through all the years and all the dictionaries in each year
Our result, movies_list_of_dict is a simple list of all the movie dictionaries.
All we have to do then is use pd.DataFrame() to use the keys as column headers and the values as the cell values in rows.

In [270]:
movies_list_of_dict = [movie_d for movie_yr in movies for movie_d in movies[movie_yr]]
df_movies = pd.DataFrame(movies_list_of_dict)


In [271]:
# Preview df_movies:
# First three movies:
df_movies.head(3)


,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,/kvRT3GwcnqGHzPjXIVrVPhUix7Z.jpg,"[12, 28, 878]",271110,en,Captain America: Civil War,"Following the events of Age of Ultron, the col...",86.040,/rAGiXaUfPzY7CDEyNKUofk3Kw2e.jpg,2016-04-27,Captain America: Civil War,False,7.4,18129
1,False,/6t8ES1d12OzWyCGxBeDYLHoaDrT.jpg,"[28, 12, 878]",330459,en,Rogue One: A Star Wars Story,A rogue band of resistance fighters unite for ...,37.495,/5jX3p0apUG5bkMHtnKZch0xpkBS.jpg,2016-12-14,Rogue One: A Star Wars Story,False,7.5,12139
2,False,/wijltj1toRcicHlCn8Cpsp4yPXe.jpg,"[12, 16, 35, 10751]",127380,en,Finding Dory,Dory is reunited with her friends Nemo and Mar...,42.162,/3UVe8NL1E2ZdUZ9EDlKGJY5UzE.jpg,2016-06-16,Finding Dory,False,7.0,9917


In [272]:
# Last three movies:
df_movies.tail(3)


,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
2997,False,/qtJGWUTaXuPVSqJOXSWoZCqucEr.jpg,"[35, 10749]",796126,en,Bad Cupid,Archie is a God on a mission to ensure that tr...,1.400,/9nSHoZpU8jFHjKwJ895bzkjPldG.jpg,2021-02-11,Bad Cupid,False,6.0,3
2998,False,None,"[80, 18, 10749]",796124,en,Ride or Die,"Ashley, a fierce friend to Mandy, takes the ph...",31.159,/77S8LQNkB0se2nk15X1d22QV1d1.jpg,2021-02-09,Ride or Die,False,4.0,2
2999,False,None,[18],796034,en,Oklahoma Mon Amour,Rural United States and modern urban Mexico me...,0.600,None,2021-04-24,Oklahoma Mon Amour,False,5.0,1


In [273]:
# See number of rows and columns, respectively:
df_movies.shape


(3000, 14)

Are there any null values?
It looks like the only null values are for backdrop_path (suffix for movie's image URL)
In other words, no missing data for this analysis so far.

In [274]:
df_movies.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   adult              3000 non-null   bool   
 1   backdrop_path      1931 non-null   object 
 2   genre_ids          3000 non-null   object 
 3   id                 3000 non-null   int64  
 4   original_language  3000 non-null   object 
 5   original_title     3000 non-null   object 
 6   overview           3000 non-null   object 
 7   popularity         3000 non-null   float64
 8   poster_path        2843 non-null   object 
 9   release_date       3000 non-null   object 
 10  title              3000 non-null   object 
 11  video              3000 non-null   bool   
 12  vote_average       3000 non-null   float64
 13  vote_count         3000 non-null   int64  
dtypes: bool(2), float64(2), int64(2), object(8)
memory usage: 287.2+ KB


Just for fun, let's explore one movie's backdrop_path - what would we do with this field?
One of my favorite movies from the past 6 years is Get Out - so lets find it in df_movies
I accessed a subset of df_movies with .loc using a pandas series of boolean values (title contains 'Get Out')

In [275]:
get_out = df_movies[['title', 'backdrop_path']].loc[(df_movies['title'].str.contains('Get Out'))]
get_out


,title,backdrop_path
531,Get Out,/vZ7EVk7FaNEWYqlVWbFLHP8Z0LU.jpg


According to the TMDb API documentation, the backdrop_path is the ```file_path``` one uses in conjunction with a ```base_url``` and ```file_size``` to generate an image URL.  Here's the backdrop_bath for Get Out appended to https://image.tmdb.org/t/p/w500/ to retrieve the w500 size:
- https://image.tmdb.org/t/p/w500/vZ7EVk7FaNEWYqlVWbFLHP8Z0LU.jpg


- I embedded it here by using <img src="get_out.png"\>, and storing the png version of the image in the same folder with this notebook.

<img src="get_out.png">

That was fun! But we must move on. Next we'll take the movies we've already gathered, which are sorted by descending revenue for years 2016-2021, and grab more details for each movie from TMDb's movie details resource. That way, we can grab the fields we need to conduct later analysis and visualization: revenue, budget, keywords, genres, and production companies.
# Our next task is to use the method movies.details() to get the columns we want for our 3,000 titles

In [276]:
# Next, we'll create a list of movie ids we have stored in df_movies
# We'll use that list to get more detailed movie info from the tmdbv3api library's movie.details() method

# Create list of movie ids from df_movies:
movie_id_list = df_movies['id'].tolist()

# Check length of list:
len(movie_id_list)


3000

The following finds any movies that would throw an error when passed to movie.details()
It also ensures we're passing field values that exist from movie.details()
Under the hood of the tmdbv3api library we are simply using the GET method to retrieve data from the TMDb server at a specified resource.
The TMDb API behaves such that if the movie id doesn't exist in the /movie/{movie_id} resource, we'll get an error back
Let's see if any of our movie ids would throw such an error:

In [277]:
my_keys = ['id', 'title', 'release_date', 'revenue', 'budget', 'genres', 'original_language',
           'spoken_languages', 'popularity', 'production_companies', 'production_countries', 'runtime', 'keywords']

details_list = []

for id_index in movie_id_list: # For each movie id in movie_id_list, we try to get results back from movie.details(), if not, the movie id is printed
    try:
        r = movie.details(id_index)
        details_list.extend({my_key: dict(r).get(my_key)
                            for my_key in my_keys})
    except:
        print(id_index)
        continue


781739


In [278]:
# Out of curiosity, lets look at the details for this movie:
df_movies[['title', 'id', 'popularity', 'overview',
           'release_date']].loc[(df_movies['id'] == 781739)


,title,id,popularity,overview,release_date
2396,Hero,781739,0.0,Hero was diagnosed with cancer at the age of 2...,2020-12-31


In [279]:
# To identify the movie lets look at the overview more closely:
print(df_movies.loc[(df_movies['id'] == 781739), 'overview'].item())


Hero was diagnosed with cancer at the age of 22. After losing faith in himself, and getting ready to give up, he found support in a place he felt safe - The Furry Fandom. Now, Hero is getting his first fursuit. This is his story.


This turned out to be a touching documentary, and one that seems to have received a decent amount of attention, but we won't be worried about not having it in our dataset. We'll remove it from movie_id_list and re-run our error-catching code.

In [280]:
# Remove movie id from list:
movie_id_list.remove(781739)


In [281]:
# Re-run error-catching code
# Returns 0 movie ids - nice!
my_keys = ['id', 'title', 'release_date', 'revenue', 'budget', 'genres', 'original_language',
           'spoken_languages', 'popularity', 'production_companies', 'production_countries', 'runtime', 'keywords']

details_list = []

for id_index in movie_id_list:
    try:
        # movie.details() returns a tmdb object, NOT a dictionary.
        r = movie.details(id_index)
        # Extend details_list by a dictionary for each movie, where each movie dictionary contains all key-value pairs for my_keys
        details_list.extend({my_key: dict(r).get(my_key)  # the tmdb object r must be converted to a dictionary to get the value for each of my keys
                            for my_key in my_keys})
    except:
        print(id_index)
        continue


Finally, we can retrieve the data we want from movie.details()!
A way I dealt with any anticipated missing fields for each movie:
In case any of the movies are missing any of the below keys, I used dict.get(), which returns None instead of throwing an error if a given key doesn't exist

In [291]:
my_keys = ['id', 'title', 'release_date', 'revenue', 'budget', 'genres', 'original_language',
           'spoken_languages', 'popularity', 'production_companies', 'production_countries', 'runtime', 'keywords']

details_list = []
details_list.extend({my_key: dict(movie.details(id_index)).get(my_key)
                    for my_key in my_keys} for id_index in movie_id_list)

print(details_list[0])  # Preview the first movie dictionary in our list!


{'id': 271110, 'title': 'Captain America: Civil War', 'release_date': '2016-04-27', 'revenue': 1153296293, 'budget': 250000000, 'genres': [{'id': 12, 'name': 'Adventure'}, {'id': 28, 'name': 'Action'}, {'id': 878, 'name': 'Science Fiction'}], 'original_language': 'en', 'spoken_languages': [{'english_name': 'Romanian', 'iso_639_1': 'ro', 'name': 'Română'}, {'english_name': 'English', 'iso_639_1': 'en', 'name': 'English'}, {'english_name': 'German', 'iso_639_1': 'de', 'name': 'Deutsch'}, {'english_name': 'Russian', 'iso_639_1': 'ru', 'name': 'Pусский'}], 'popularity': 86.04, 'production_companies': [{'id': 420, 'logo_path': '/hUzeosd33nzE5MCNsZxCGEKTXaQ.png', 'name': 'Marvel Studios', 'origin_country': 'US'}], 'production_countries': [{'iso_3166_1': 'US', 'name': 'United States of America'}], 'runtime': 147, 'keywords': {'keywords': [{'id': 393, 'name': 'civil war'}, {'id': 9663, 'name': 'sequel'}, {'id': 9715, 'name': 'superhero'}, {'id': 9717, 'name': 'based on comic'}, {'id': 179430, 

In [292]:
# Now we take this list of dictionaries and easily convert it to a pandas dataframe df_details:
df_details = pd.DataFrame(details_list)
df_details[0:3] # Preview the first three movies


,id,title,release_date,revenue,budget,genres,original_language,spoken_languages,popularity,production_companies,production_countries,runtime,keywords
0,271110,Captain America: Civil War,2016-04-27,1153296293,250000000,"[[id, name], [id, name], [id, name]]",en,"[[english_name, iso_639_1, name], [english_nam...",86.040,"[[id, logo_path, name, origin_country]]","[[iso_3166_1, name]]",147.0,[keywords]
1,330459,Rogue One: A Star Wars Story,2016-12-14,1056057273,200000000,"[[id, name], [id, name], [id, name]]",en,"[[english_name, iso_639_1, name]]",37.495,"[[id, logo_path, name, origin_country]]","[[iso_3166_1, name]]",133.0,[keywords]
2,127380,Finding Dory,2016-06-16,1028570889,200000000,"[[id, name], [id, name], [id, name], [id, name]]",en,"[[english_name, iso_639_1, name]]",42.162,"[[id, logo_path, name, origin_country]]","[[iso_3166_1, name]]",97.0,[keywords]


# We finally have the data we want stored in df_details.

Take a look at big-picture of df_details
It makes sense that we have 2999 entries since the movie Hero doesn't exist in movie.details()
It looks like we have one null value for run-time

In [293]:
df_details.index
df_details.info()
df_details.columns
df_details.dtypes


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    2999 non-null   int64  
 1   title                 2999 non-null   object 
 2   release_date          2999 non-null   object 
 3   revenue               2999 non-null   int64  
 4   budget                2999 non-null   int64  
 5   genres                2999 non-null   object 
 6   original_language     2999 non-null   object 
 7   spoken_languages      2999 non-null   object 
 8   popularity            2999 non-null   float64
 9   production_companies  2999 non-null   object 
 10  production_countries  2999 non-null   object 
 11  runtime               2998 non-null   float64
 12  keywords              2999 non-null   object 
dtypes: float64(2), int64(3), object(8)
memory usage: 304.7+ KB


id                        int64
title                    object
release_date             object
revenue                   int64
budget                    int64
genres                   object
original_language        object
spoken_languages         object
popularity              float64
production_companies     object
production_countries     object
runtime                 float64
keywords                 object
dtype: object

# My first data-cleaning task was to detect any movies not originally shot in English
- Since our studio is still in R&D mode, we'll focus on movies originally shot in English
- 'with_original_language' was already set to 'en' upon intake, but this next step helps find foreign language films that slipped through


In [ ]:
# Detect non-english strings in title and remove them from df_details:

pd.set_option('display.max_rows', df_details.shape[0]+1) # Set display option so that we can see all rows because we will likely manually review the values:

# Create function to detect non-english strings:
def detect_en(ascii_string):
    try:
        ascii_string.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

column_names = ["id", "title"]
flag_non_en = pd.DataFrame(columns = column_names)
for item in df_details.loc[:, 'title']:
    if isEnglish(item) == False:
        flag_non_en = flag_non_en.append(df_details[['id','title']].loc[df_details['title'] == item])
# print(flag_non_en)


I made a list of the movies I wanted to remove from df_details, based on internet research to see if original language is English.

- 720980      Express Yourself: Die Tänzer der Queen of Pop
- 676624                                               عتيج
- 671272                              Óleos en la oscuridad
- 664648                                          Crisálida
- 692856                                              Avīci
- 691836                      Avante (Ao Vivo em São Paulo)
- 680817                                      Hranice práce
- 676912                                            Sátiros
- 703348                                               عقلي
- 720570                                   maníaco da ponte
- 832125                                  Ghostemane: Ímpio
- 806831                 História Secreta do Pop Brasileiro
- 783999        Crimes de haine en Amérique : l'inquiétante
- 740354                                          Bach-Hông
- 793311                                   Toilette d’amour
- 787619                                           Czyściec
- 832310                                           Verrückt
- 822161                                         ¿Libertad?
- 802434                                              Acéré

In [316]:
# Remove rows with these ids from df_details:
non_eng_ids = [720980, 676624, 671272, 664648, 692856, 691836, 680817, 676912, 703348, 720570, 832125, 806831, 783999, 740354, 793311, 787619, 832310, 822161, 802434]

# Remove movie id from list:
df_details = df_details[~df_details['id'].isin(non_eng_ids)]


In [317]:
# Check to make sure they're no longer in there:
df_details[df_details['id'].isin(non_eng_ids)]

,id,title,release_date,revenue,budget,genres,original_language,spoken_languages,popularity,production_companies,production_countries,runtime,keywords,genres_list_dict,genres_list_str


# The next data cleaning step was to convert the date field from str to datetime64
A lot of the fields we got back are objects.
Lets make sure that release_date is an object we can work with
We'll use the first movie in our list as an example:
271110	Captain America: Civil War

In [296]:
capt_amer_date = df_details.loc[(
    df_details['id'] == 271110), 'release_date'].item()
print(type(capt_amer_date))
print(capt_amer_date)


<class 'str'>
2016-04-27


Since it's a string, lets convert it to dtype ```datetime64[ns]``` which is built on the NumPy ```datetime64``` dtype

In [297]:
df_details['release_date'] = pd.to_datetime(df_details['release_date'])

capt_amer_date = df_details.loc[(
    df_details['id'] == 271110), 'release_date'].item()
print(type(capt_amer_date))
print(capt_amer_date)


<class 'pandas._libs.tslibs.timestamps.Timestamp'>
2016-04-27 00:00:00


# The final data cleaning step was to convert fields that are lists of objects to more simple datastructures:
I'll need to convert any field that I want to use in a visualization into a useable format. Let's start with genres!

In [343]:
# Preview the genres column, dtype object
my_sample = df_details.loc[0:2, 'genres']
type(my_sample)
my_sample
for mystery in my_sample:
    print(type(mystery))
    print(type(mystery[0]))
    print(mystery)


<class 'list'>
<class 'tmdbv3api.as_obj.AsObj'>
[{'id': 12, 'name': 'Adventure'}, {'id': 28, 'name': 'Action'}, {'id': 878, 'name': 'Science Fiction'}]
<class 'list'>
<class 'tmdbv3api.as_obj.AsObj'>
[{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 878, 'name': 'Science Fiction'}]
<class 'list'>
<class 'tmdbv3api.as_obj.AsObj'>
[{'id': 12, 'name': 'Adventure'}, {'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]


In [ ]:
# Convert these lists of objects to lists of dictionaries using .apply() to map each genre value
def convert_to_list_of_dict(val):
    # val parameter is used to transform each value of the pandas genres series
    val = [dict(x) for x in val]
    return val


df_details['genres_list_dict'] = df_details.loc[:,
                                                'genres'].apply(convert_to_list_of_dict)
                                                

In [342]:
my_sample = df_details.loc[0:2, 'genres_list_dict']
for mystery in my_sample:
    print(type(mystery))
    print(type(mystery[0]))
    print(mystery)


<class 'list'>
<class 'dict'>
[{'id': 12, 'name': 'Adventure'}, {'id': 28, 'name': 'Action'}, {'id': 878, 'name': 'Science Fiction'}]
<class 'list'>
<class 'dict'>
[{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 878, 'name': 'Science Fiction'}]
<class 'list'>
<class 'dict'>
[{'id': 12, 'name': 'Adventure'}, {'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]


In [300]:
# Preview our new column "genres_list_dict"
df_details['genres_list_dict'][0]

[{'id': 12, 'name': 'Adventure'},
 {'id': 28, 'name': 'Action'},
 {'id': 878, 'name': 'Science Fiction'}]

In [ ]:
# Convert lists of dictionaries to a list of strings, where each string is the name of a genre
def convert_to_list_of_str(val):
    # val parameter is used to transform each value of the pandas genres series
    val = [x.get('name') for x in val]
    return val


df_details['genres_list_str'] = df_details.loc[:,
                                                'genres_list_dict'].apply(convert_to_list_of_str)
                                                

In [341]:
my_sample = df_details.loc[0:2, 'genres_list_str']
for mystery in my_sample:
    print(type(mystery))
    print(type(mystery[0]))
    print(mystery)


<class 'list'>
<class 'str'>
['Adventure', 'Action', 'Science Fiction']
<class 'list'>
<class 'str'>
['Action', 'Adventure', 'Science Fiction']
<class 'list'>
<class 'str'>
['Adventure', 'Animation', 'Comedy', 'Family']


In [339]:
# Preview the results:
df_details[['title', 'genres_list_str']].iloc[400:403]

,title,genres_list_str
401,The Last Appeal,"[Drama, TV Movie]"
402,Cinema Self,NaN
403,Farewell Boleyn,[Documentary]


In [307]:
# Rather than empty lists, I want movies with no genre to be the numpy null value, which is NaN:
import numpy as np
df_details['genres_list_str'] = df_details['genres_list_str'].apply(
    lambda x: np.nan if len(x) == 0 else x)


In [315]:
# Check that it worked:
is_NaN = df_details.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = df_details[row_has_NaN]

rows_with_NaN[['title', 'genres','genres_list_str']].iloc[0:3]


,title,genres,genres_list_str
276,The Girl on the Mat,[],NaN
302,First Day Out,[],NaN
303,Wild Flowers,[],NaN


# Now we're going to do the same thing for production_companies - it has the same structure as genres:

In [336]:
my_sample = df_details.loc[0:2, 'production_companies']
for mystery in my_sample:
    print(type(mystery))
    print(type(mystery[0]))
    print(mystery)

<class 'list'>
<class 'tmdbv3api.as_obj.AsObj'>
[{'id': 420, 'logo_path': '/hUzeosd33nzE5MCNsZxCGEKTXaQ.png', 'name': 'Marvel Studios', 'origin_country': 'US'}]
<class 'list'>
<class 'tmdbv3api.as_obj.AsObj'>
[{'id': 1, 'logo_path': '/o86DbpburjxrqAzEDhXZcyE8pDb.png', 'name': 'Lucasfilm Ltd.', 'origin_country': 'US'}]
<class 'list'>
<class 'tmdbv3api.as_obj.AsObj'>
[{'id': 3, 'logo_path': '/1TjvGVDMYsj6JBxOAkUHpPEwLf7.png', 'name': 'Pixar', 'origin_country': 'US'}]


In [ ]:
def convert_to_list_of_dict(val):
    val = [dict(x) for x in val]
    return val


df_details['prod_list_dict'] = df_details.loc[:,
                                                'production_companies'].apply(convert_to_list_of_dict)
                                                

In [340]:
my_sample = df_details.loc[0:2, 'prod_list_dict']
for mystery in my_sample:
    print(type(mystery))
    print(type(mystery[0]))
    print(mystery)


<class 'list'>
<class 'dict'>
[{'id': 420, 'logo_path': '/hUzeosd33nzE5MCNsZxCGEKTXaQ.png', 'name': 'Marvel Studios', 'origin_country': 'US'}]
<class 'list'>
<class 'dict'>
[{'id': 1, 'logo_path': '/o86DbpburjxrqAzEDhXZcyE8pDb.png', 'name': 'Lucasfilm Ltd.', 'origin_country': 'US'}]
<class 'list'>
<class 'dict'>
[{'id': 3, 'logo_path': '/1TjvGVDMYsj6JBxOAkUHpPEwLf7.png', 'name': 'Pixar', 'origin_country': 'US'}]


In [338]:
# Preview our new column "prod_list_dict"
df_details['prod_list_dict'][0]

[{'id': 420,
  'logo_path': '/hUzeosd33nzE5MCNsZxCGEKTXaQ.png',
  'name': 'Marvel Studios',
  'origin_country': 'US'}]

In [ ]:
def convert_to_list_of_str(val):
    val = [x.get('name') for x in val]
    return val


df_details['prod_list_str'] = df_details.loc[:,
                                                'prod_list_dict'].apply(convert_to_list_of_str)
                                                

In [345]:
my_sample = df_details.loc[0:2, 'prod_list_str']
for mystery in my_sample:
    if len(mystery) == 0:
        print(mystery)


<class 'list'>
<class 'str'>
['Marvel Studios']
<class 'list'>
<class 'str'>
['Lucasfilm Ltd.']
<class 'list'>
<class 'str'>
['Pixar']


In [ ]:
# Make empty lists NaN:
import numpy as np
df_details['prod_list_nan'] = df_details['prod_list_str'].apply(
    lambda x: np.nan if len(x) == 0 else x)

In [349]:
# Check that it worked:
is_NaN = df_details.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = df_details[row_has_NaN]

rows_with_NaN[['title', 'genres_list_str','prod_list_str']].iloc[0:3]

,title,genres_list_str,prod_list_str
31,Split,"[Fantasy, Drama, Thriller, Crime]",NaN
201,Hillsong: Let Hope Rise,[Documentary],NaN
228,Lulu the Movie,[Comedy],NaN


# Finally, I created a dataframe with only the columns I wanted to keep:

In [350]:
# Keep these Columns:
df = df_details[['id', 'title', 'release_date', 'revenue', 'budget',
                 'genres_list_str', 'popularity', 'prod_list_str', 'runtime']].copy()


In [351]:
df.head()

,id,title,release_date,revenue,budget,genres_list_str,popularity,prod_list_str,runtime
0,271110,Captain America: Civil War,2016-04-27,1153296293,250000000,"[Adventure, Action, Science Fiction]",86.040,[Marvel Studios],147.0
1,330459,Rogue One: A Star Wars Story,2016-12-14,1056057273,200000000,"[Action, Adventure, Science Fiction]",37.495,[Lucasfilm Ltd.],133.0
2,127380,Finding Dory,2016-06-16,1028570889,200000000,"[Adventure, Animation, Comedy, Family]",42.162,[Pixar],97.0
3,269149,Zootopia,2016-02-11,1023784195,150000000,"[Animation, Adventure, Family, Comedy]",60.016,"[Walt Disney Animation Studios, Walt Disney Pi...",109.0
4,278927,The Jungle Book,2016-04-07,966550600,175000000,"[Family, Adventure, Drama, Fantasy]",34.177,"[Walt Disney Pictures, Fairview Entertainment,...",106.0


In [ ]:
df.